In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import traceback

import torch

from sem2SQL import transform
from src import args as arg
from src import utils
from src.models.model import IRNet
from src.rule import semQL

import subprocess

from dummy_input import DUMMY_INPUT
from src.rule.sem_utils import alter_column0, alter_inter, alter_not_in
from src.utils import load_data_new

MODEL_PATH = "./saved_model/IRNet_pretrained.model"
EMBEDS_PATH = "./data/glove.42B.300d.txt"
QUESTION_DATA_PATH = "./data/custom/question.json"
TABLE_DATA_PATH = "./data/custom/tables.json"
PREDICT_LF_PATH = "./data/custom/predict_lf.json"

CUDA = False

In [3]:
arg_parser = arg.init_arg_parser()
args = arg_parser.parse_args("--dataset ./data/custom --glove_embed_path ./data/glove.42B.300d.txt --cuda --epoch 50 --loss_epoch_threshold 50 --sketch_loss_coefficie 1.0 --beam_size 5 --seed 90 --save ${save_name} --embed_size 300 --sentence_features --column_pointer --hidden_size 300 --lr_scheduler --lr_scheduler_gammar 0.5 --att_vec_size 300 --load_model ./saved_model/IRNet_pretrained.model".split())

In [4]:
grammar = semQL.Grammar()
model = IRNet(args, grammar)

if CUDA: model.cuda()

print('load pretrained model from %s' % MODEL_PATH)
pretrained_model = torch.load(MODEL_PATH,
                              map_location=lambda storage, loc: storage)
import copy
pretrained_modeled = copy.deepcopy(pretrained_model)
for k in pretrained_model.keys():
    if k not in model.state_dict().keys():
        del pretrained_modeled[k]

model.load_state_dict(pretrained_modeled)

model.word_emb = utils.load_word_emb(EMBEDS_PATH)

model.eval()

Use Column Pointer:  True
load pretrained model from ./saved_model/IRNet_pretrained.model
Loading word embedding from ./data/glove.42B.300d.txt


IRNet(
  (encoder_lstm): LSTM(300, 150, batch_first=True, bidirectional=True)
  (lf_decoder_lstm): LSTMCell(556, 300)
  (sketch_decoder_lstm): LSTMCell(556, 300)
  (decoder_cell_init): Linear(in_features=300, out_features=300, bias=True)
  (att_sketch_linear): Linear(in_features=300, out_features=300, bias=False)
  (att_lf_linear): Linear(in_features=300, out_features=300, bias=False)
  (sketch_att_vec_linear): Linear(in_features=600, out_features=300, bias=False)
  (lf_att_vec_linear): Linear(in_features=600, out_features=300, bias=False)
  (prob_att): Linear(in_features=300, out_features=1, bias=True)
  (prob_len): Linear(in_features=1, out_features=1, bias=True)
  (col_type): Linear(in_features=4, out_features=300, bias=True)
  (sketch_encoder): LSTM(128, 64, batch_first=True, bidirectional=True)
  (production_embed): Embedding(46, 128)
  (type_embed): Embedding(10, 128)
  (att_project): Linear(in_features=428, out_features=300, bias=True)
  (N_embed): Embedding(5, 128)
  (query_vec

In [13]:
question_str = "How many customers do we have?"

In [14]:
DUMMY_INPUT["question"] = question_str
DUMMY_INPUT["question_toks"] = question_str.split(" ")

import json
with open(QUESTION_DATA_PATH, 'w', encoding='utf-8') as f:
    json.dump([DUMMY_INPUT], f, ensure_ascii=False, indent=4)

subprocess.run(["cd preprocess && bash run_me.sh ../data/custom/question.json ../data/custom/tables.json"], shell=True)

with open(TABLE_DATA_PATH) as inf:
    print("Loading data from %s" % TABLE_DATA_PATH)
    table_data = json.load(inf)

Loading data from ./data/custom/tables.json


In [20]:
question_data, table_data_new = load_data_new(QUESTION_DATA_PATH, table_data, use_small=False)

Loading data from ./data/custom/question.json


In [21]:
question_data

[{'db_id': 'customer',
  'query': 'SELECT count(*) FROM customer',
  'query_toks': ['SELECT', 'count', '(', '*', ')', 'FROM', 'customer'],
  'query_toks_no_value': ['select',
   'count',
   '(',
   '*',
   ')',
   'from',
   'customer'],
  'question': 'How many customers do we have?',
  'question_toks': ['how', 'many', 'customer', 'do', 'we', 'have?'],
  'sql': {'except': None,
   'from': {'conds': [], 'table_units': [['table_unit', 0]]},
   'groupby': [],
   'having': [],
   'intersect': None,
   'limit': None,
   'orderby': [],
   'select': [False, [[3, [0, [0, 0, False], None]]]],
   'union': None,
   'where': []},
  'names': ['*',
   'customer_Id',
   'name',
   'address',
   'birthday',
   'phone',
   'city'],
  'table_names': ['customer'],
  'col_set': ['*',
   'customer_Id',
   'name',
   'address',
   'birthday',
   'phone',
   'city'],
  'col_table': [-1, 0, 0, 0, 0, 0, 0],
  'keys': {'1': 1},
  'origin_question_toks': ['How', 'many', 'customers', 'do', 'we', 'have?'],
  'ques

In [17]:
table_data_new

{'customer': {'column_names': [[-1, '*'],
   [0, 'customer_Id'],
   [0, 'name'],
   [0, 'address'],
   [0, 'birthday'],
   [0, 'phone'],
   [0, 'city']],
  'column_names_original': [[-1, '*'],
   [0, 'customer_ID'],
   [0, 'name'],
   [0, 'address'],
   [0, 'birthday'],
   [0, 'phone'],
   [0, 'city']],
  'column_types': ['text', 'number', 'text', 'text', 'text', 'number', 'text'],
  'db_id': 'customer',
  'foreign_keys': [],
  'primary_keys': [1],
  'table_names': ['customer'],
  'table_names_original': ['customer']}}

In [22]:
json_datas = utils.get_json_data(model, table_data_new, question_data, beam_size=args.beam_size)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx